# Worksheet 08

Name:  Jackson Fisk
UID:   U59841760

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [14]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

c1 = np.random.normal(mean, stdev, 10).tolist()
print(c1)

[6.106538841201122, 4.60223258272694, 3.888411721621007, 3.891910448806363, 4.018509459979799, 5.633782060625073, 4.503027293159177, 3.7230884404787363, 5.468485185425828, 4.2131082337173815]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [15]:
c2 = np.random.normal(8,1,10).tolist()
print(c2)

[7.641394657685791, 8.123308996708992, 7.5998967921846345, 8.784146459305914, 7.270185182895268, 8.077573348313859, 7.771986679283963, 5.773968530388815, 8.636836280820258, 8.0766728991467]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [16]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = c1.pop()
        data.append(p1)
    else:
        p2 = c2.pop()
        data.append(p2)
print(data)

[8.0766728991467, 8.636836280820258, 5.773968530388815, 7.771986679283963, 4.2131082337173815, 8.077573348313859, 7.270185182895268, 8.784146459305914, 7.5998967921846345, 8.123308996708992]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

It's a Gaussian mixture model with 2 mixture components
K = 2 
pi = .5
We need to estimate mean and covariance matrices.


d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(C_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(C_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [17]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

c1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(c1)
c2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(c2)

prob_c = [ len(c1) / (len(c1) + len(c2)) , len(c2) / (len(c1) + len(c2))]
mean = [ sum(c1)/len(c1) , sum(c2) / len(c2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, c1)) / len(c1) , sum(map(lambda x: (x - mean[1]) ** 2, c2)) / len(c2) ]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[8.0766728991467, 8.636836280820258, 7.771986679283963, 8.077573348313859, 7.270185182895268, 8.784146459305914, 7.5998967921846345, 8.123308996708992]
[5.773968530388815, 4.2131082337173815]
P(C_1) = 0.8,  P(C_2) = 0.2
mean_1 = 8.04257582983245,  mean_2 = 4.993538382053098
var_1 = 0.2222185366273602,  var_2 = 0.6090712164313086


e) For each data point, compute `P(C_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [18]:
from scipy.stats import norm

prob_c0_x = [] # P(C_0 | X_i)
prob_c1_x = [] # P(C_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | C_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(C_j) already computed
        prob_c[j]

    # P(X_i) = P(C_0)P(X_i | C_0) + P(C_1)P(X_i | C_1)
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    # P(C_j | X_i) = P(X_i | C_j)P(C_j) / P(X_i)
    prob_c0_x_i = (pdf_i[0] * prob_c[0]) / prob_x
    prob_c1_x_i = (pdf_i[1] * prob_c[1]) / prob_x

    prob_c0_x.append(prob_c0_x_i)
    prob_c1_x.append(prob_c1_x_i)

probs = zip(data, prob_c0_x, prob_c1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from C_1 = " + str(p[1]))
    print("Probability of coming from C_2 = " + str(p[2]))
    print()


point =  8.0766728991467
probability of observing that point if it came from cluster 0 =  1.7742603481050947
probability of observing that point if it came from cluster 1 =  1.7866403630154604e-06
point =  8.636836280820258
probability of observing that point if it came from cluster 0 =  0.05025922436438356
probability of observing that point if it came from cluster 1 =  1.112969625018389e-08
point =  5.773968530388815
probability of observing that point if it came from cluster 0 =  4.194486359297036e-23
probability of observing that point if it came from cluster 1 =  0.28821731458423494
point =  7.771986679283963
probability of observing that point if it came from cluster 0 =  0.8553826749481045
probability of observing that point if it came from cluster 1 =  1.9835597678872908e-05
point =  4.2131082337173815
probability of observing that point if it came from cluster 0 =  5.852902764738405e-65
probability of observing that point if it came from cluster 1 =  0.2882173145842355
point =

f) Having computed `P(C_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(C_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [20]:
prob_c = [sum(prob_c0_x) / len(prob_c0_x), sum(prob_c1_x) / len(prob_c1_x)]
mean = [sum([x[0] * x[1] for x in zip(prob_c0_x, data)]) / sum(prob_c0_x),
        sum([x[0] * x[1] for x in zip(prob_c1_x, data)]) / sum(prob_c1_x)]

var = [sum([x[0] * (x[1] - mean[0]) ** 2 for x in zip(prob_c0_x, data)]) / sum(prob_c0_x),
       sum([x[0] * (x[1] - mean[1]) ** 2 for x in zip(prob_c1_x, data)]) / sum(prob_c1_x)]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))


P(C_1) = 0.7965694756364133,  P(C_2) = 0.20343052436358663
mean_1 = 8.04589886334999,  mean_2 = 5.031943484767878
var_1 = 0.22059918576528678,  var_2 = 0.6847949347872997


g) Update `P(C_j | X_i)`. Comment on any differences or lack thereof you observe.

In [24]:
from scipy.stats import norm

prob_c0_x = []  # P(C_0 | X_i)
prob_c1_x = []  # P(C_1 | X_i)
prob_x = []  # P(X_i)

k = 2

for p in data:
    pdf_i = []

    for j in range(k):
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        prob_c[j]
    prob_x_i = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]
    prob_x.append(prob_x_i)

    prob_c0_x_i = pdf_i[0] * prob_c[0] / prob_x_i
    prob_c1_x_i = pdf_i[1] * prob_c[1] / prob_x_i

    prob_c0_x.append(prob_c0_x_i)
    prob_c1_x.append(prob_c1_x_i)

probs = zip(data, prob_c0_x, prob_c1_x)
for p in probs:
    print("Data Point =", p[0])
    print("Probability of coming from C_0 =", p[1])
    print("Probability of coming from C_1 =", p[2])
    print()


Data Point = 8.0766728991467
Probability of coming from C_0 = 0.999995765922359
Probability of coming from C_1 = 4.234077640999658e-06

Data Point = 8.636836280820258
Probability of coming from C_0 = 0.9999971428597743
Probability of coming from C_1 = 2.857140225831975e-06

Data Point = 5.773968530388815
Probability of coming from C_0 = 2.029853685591542e-22
Probability of coming from C_1 = 1.0

Data Point = 7.771986679283963
Probability of coming from C_0 = 0.9999406458727538
Probability of coming from C_1 = 5.935412724623633e-05

Data Point = 4.2131082337173815
Probability of coming from C_0 = 6.992540004316503e-65
Probability of coming from C_1 = 1.0

Data Point = 8.077573348313859
Probability of coming from C_0 = 0.9999957881749998
Probability of coming from C_1 = 4.211825000116572e-06

Data Point = 7.270185182895268
Probability of coming from C_0 = 0.8398004117172668
Probability of coming from C_1 = 0.1601995882827331

Data Point = 8.784146459305914
Probability of coming from C_0 

h) Use `P(C_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [25]:
# Create an empty list to store the hard assignments
hard_assignments = []

# Assign each data point to a cluster based on the higher probability
for p in zip(prob_c0_x, prob_c1_x):
    if p[0] > p[1]:
        hard_assignments.append(0)
    else:
        hard_assignments.append(1)

# Now, the 'hard_assignments' list contains the cluster assignments (0 or 1) for each data point


### Clustering Aggregation

| Point | C | P |
|-------|---|---|
| A     | 0 | a |
| B     | 0 | b |
| C     | 2 | b |
| D     | 1 | c |
| E     | 1 | d |

a) Fill in the following table where for each pair of points determine whether C and P agree or disagree on how to cluster that pair.

| Pair | Disagreement |
|------|--------------|
| A  B |      1       |
| A  C |      0       |
| A  D |      1       |
| A  E |      1       |
| B  C |      1       |
| B  D |      1       |
| B  E |      1       |
| C  D |      1       |
| C  E |      1       |
| D  E |      0       |


As datasets become very large, this process can become computationally challenging.

b) Given N points, what is the formula for the number of unique pairs of points one can create?

(N choose 2)

Assume that clustering C clusters all points in the same cluster and clustering P clusters points as such:

| Point | P |
|-------|---|
| A     | 0 |
| B     | 0 |
| C     | 0 |
| D     | 1 |
| E     | 1 |
| F     | 2 |
| G     | 2 |
| H     | 2 |
| I     | 2 |

c) What is the maximum number of disagreements there could be for a dataset of this size? (use the formula from b)?

(9 choose 2) = 9! / (2! * (9-2)!) = 36 -3 = 33

d) If we look at cluster 0. There are (3 x 2) / 2 = 3 pairs that agree with C (since all points in C are in the same cluster). For each cluster, determine how many agreements there are. How many total agreements are there? How many disagreements does that mean there are between C and P?

32

e) Assuming that filtering the dataset by cluster number is a computationally easy operation, describe an algorithm inspired by the above process that can efficiently compute disagreement distances on large datasets.

1 Group the points in the dataset by cluster number.
2 For each pair of clusters, count the number of pairs of points that belong to different clusters in each partition.
3 For each cluster, count the number of pairs of points that belong to different partitions.
4 Add up the counts from steps 2 and 3 to get the total number of disagreements between partitions C and P.